In [27]:
%matplotlib notebook
%run models.ipynb
%run preprocess.ipynb

import os
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.utils.data as td 
import torchvision as tv
from PIL import Image
import matplotlib.pyplot as plt
import nntools as nt
import PIL
import math
from torch.autograd import Variable

In [21]:
def train_model(model, data, epochs, B=100, gamma=0.01, rho=0.9):
    model.train()
    
    N = len(data) # size of data
    NB = int((N+B-1)/B) # number of minibatches
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=gamma, momentum=rho)
    
    for t in epochs:
        running_loss = 0
        shuffled_indices = np.random.permutation(range(N))
        
        for k in range(NB):
            # extract k-th minibatch from xtrain and ltrain
            minibatch_indices = shuffled_indices[B*k:min(B*(k+1), N)]
            
            print(minibatch_indices)
            
            break
        
        break
            

In [26]:
train

<function __main__.train(model, data, epochs, B=100, gamma=0.01, rho=0.9)>

torch.Size([1, 1, 1024])

In [20]:
train[(0, 1)]

TypeError: list indices must be integers or slices, not tuple